# 04 RAG | 02 Embeddings | 04 Text Embeddings Extensions

## Azure Environment

Necessary parameter are imported from [./Configuration/application.env]. Check [Create Environment](../../01_CreateEnvironment/01_Environment.ipynb) to setup the necessary demo environment.


## Step 1: EmbeddingGenerator

The interface `IEmbeddingGenerator` is provided within the nuget package `Microsoft.Extensions.AI.OpenAI` and delivers the intended abstraction.

The `AzureOpenAIClient` from the Azure.AI.OpenAI NuGet package offers a unified interface to create various specialized clients, each designed to handle specific tasks. The `EmbeddingClient` is used to create multi dimensional vectors from provided text and casted to an `IEmbeddingGenerator` instance using the `AsEmbeddingGenerator()` method provided by `Microsoft.Extensions.AI.OpenAI`


In [ ]:
#r "nuget: Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.Abstractions, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.3.0-preview.1.25161.3"
#r "nuget: Azure.AI.OpenAI, 2.2.0-beta.2"
#r "nuget: DotNetEnv, 3.1.1"

using System.ClientModel;
using Microsoft.Extensions.AI;
using OpenAI.Embeddings;
using Azure.AI.OpenAI;

static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string assetsFolder = Environment.GetEnvironmentVariable("WS_ASSETS_FOLDER") ?? "WS_ASSETS_FOLDER not found";;

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";    
string embeddingDeploymentName = Environment.GetEnvironmentVariable("WS_EMBEDDING_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";

ApiKeyCredential azureKeyCredential = new ApiKeyCredential(oAiApiKey);
EmbeddingClient embeddingClient = new EmbeddingClient(embeddingDeploymentName, oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);

IEmbeddingGenerator<string, Embedding<float>>  embeddingGenerator = 
    azureOpenAIClient.AsEmbeddingGenerator(embeddingDeploymentName);

Console.WriteLine($"EmbeddingGenerator created ...");


## Step 2: Create Embeddings

Embeddings will be created from the following text resources: 

- [WikiAKS.txt](../../Assets/Embedding/WikiAks.txt): A download from the MS Azure Kubernetes Service documentation
- [WikiSuperBowl2024](../../Assets/Embedding/WikiSuperBowl2024.txt): A download from Wikipedia with information about the 2024 Super Bowl
- The string literal: *Who won the Super Bowl in 2024* 
- The string literal: *The Kansas City Chiefs won the Super Bowl in 2024*

The function `GenerateAsync()` provided by `IEmbeddingGenerator` is used to create the embeddings by calling the model deployment.

In [13]:

// Vectorize AKS (Azure Kubernetes Service) information from Wikipedia
string documentationPage = Path.Combine(assetsFolder,"Embedding", "WikiAks.txt");
string textToBeVectorized = File.ReadAllText(documentationPage);

float[] embeddingWikiAks = Array.Empty<float>();
GeneratedEmbeddings<Embedding<float>> embedding = await embeddingGenerator.GenerateAsync(
    new string[] { textToBeVectorized }
);
embeddingWikiAks = embedding[0].Vector.ToArray();
Console.WriteLine($"Vector wiki info AKS created...");

// Vectorize Super Bowl information from Wikipedia
documentationPage = Path.Combine(assetsFolder,"Embedding", "WikiSuperBowl2024.txt");
textToBeVectorized = File.ReadAllText(documentationPage);

float[] embeddingWikiSuperBowl = Array.Empty<float>();
embedding = await embeddingGenerator.GenerateAsync(
    new string[] { textToBeVectorized }
);
embeddingWikiSuperBowl = embedding[0].Vector.ToArray();
Console.WriteLine($"Vector wiki info Super Bowl created...");

// Vectorize a question
textToBeVectorized = "Who won the Super Bowl in 2024";
float[] embeddingQuestion = Array.Empty<float>();
embedding = await embeddingGenerator.GenerateAsync(
    new string[] { textToBeVectorized }
);
embeddingQuestion = embedding[0].Vector.ToArray();

Console.WriteLine($"Vector question Super Bowl created...");

// Vectorize a statement
textToBeVectorized = "The Kansas City Chiefs won the Super Bowl in 2024";
float[] embeddingStatement = Array.Empty<float>();
embedding = await embeddingGenerator.GenerateAsync(
    new string[] { textToBeVectorized }
);
embeddingStatement = embedding[0].Vector.ToArray();

Console.WriteLine($"Vector statement Super Bowl created...");


Vector wiki info AKS created...
Vector wiki info Super Bowl created...
Vector question Super Bowl created...
Vector statement Super Bowl created...


## Step 3: Vector storage

All four created vectors are stored in a txt file for analyze and further usage in other notebooks.

In [14]:
using System.Linq;

string wikiAKSFileName = Path.Combine(assetsFolder, "Embedding", "TextEmbedding_WikiAks.txt");
string wikiSuperBowlFileName = Path.Combine(assetsFolder, "Embedding", "TextEmbedding_WikiSuperBowl.txt");
string queryVectorFileName = Path.Combine(assetsFolder, "Embedding", "TextEmbedding_Query.txt");
string statementVectorFileName = Path.Combine(assetsFolder, "Embedding", "TextEmbedding_Statement.txt");

await File.WriteAllLinesAsync(
    wikiAKSFileName, 
    embeddingWikiAks.Select(vector => vector.ToString())
);

await File.WriteAllLinesAsync(
    wikiSuperBowlFileName, 
    embeddingWikiSuperBowl.Select(vector => vector.ToString())
);

await File.WriteAllLinesAsync(
    queryVectorFileName, 
    embeddingQuestion.Select(vector => vector.ToString())
);

await File.WriteAllLinesAsync(
    statementVectorFileName, 
    embeddingStatement.Select(vector => vector.ToString())
);


Console.WriteLine($"AKS vector stored in {wikiAKSFileName}...");
Console.WriteLine($"Super Bowl vector stored in {wikiSuperBowlFileName}...");
Console.WriteLine($"Query Vector stored in {queryVectorFileName}...");
Console.WriteLine($"Statement Vector stored in {statementVectorFileName}...");


AKS vector stored in ../../assets\Embedding\TextEmbedding_WikiAks.txt...
Super Bowl vector stored in ../../assets\Embedding\TextEmbedding_WikiSuperBowl.txt...
Query Vector stored in ../../assets\Embedding\TextEmbedding_Query.txt...
Statement Vector stored in ../../assets\Embedding\TextEmbedding_Statement.txt...
